In [ ]:
import torch #导入pytorch库
import numpy as np #导入numpy库
import random #导入random库


In [ ]:
#此模块用于生成虚拟数据
def synthetic_data(w, b ,num_examples):#定义一个合成数据的函数用于生成虚拟数据以便于做模型测试训练
    X = torch.normal(0, 1, (num_examples, len(w)))#定义 X 为均值为0方差为1的随机数，尺寸为样本个数行，偏置长度列
    y = torch.matmul(X, w) + b#定义 y 为 X 和 w 的内积加上 b
    y += torch.normal(0, 0.01, y.shape)#用给 y 累加的形式，数学上表示为 y = dot(X, w) + b + noise,定义噪声为均值为0方差为0.01的随机数，]
    return X, y.reshape(-1, 1)#将 y 转化为(num_examples, 1),方便后续计算

true_w = torch.tensor([2,-3.4])#定义一个明确的 w
true_b = 4.2#定义一个明确的偏置 b
features,labels = synthetic_data(true_w, true_b, 1000)#将 X，y 写作特征和标签，以方便以后调用 

X,y = synthetic_data(true_w, true_b, 1000)

In [ ]:
#批量数据处理
def data_iter(batch_size, features, labels):#定义一个数据迭代器
    num_examples = len(features)#函数内定义样本数量等于特征数量
    indices = list(range( num_examples))#将样本数量转化为列表并且定义出索引
    random.shuffle(indices)#打乱索引
    for i in range(0, num_examples, batch_size):#对于整个样本集进行一个小批量抽取的遍历
        batch_indices = torch.tensor(indices[i:min(i + batch_size, num_examples)])#小批量抽取的主体，为了防止 i + batch_size 大于整个样本集使用了 min 函数确保不会取值溢出
        yield features[batch_indices], labels[batch_indices]#返回抽取小批量样本的特征和标签

batch_size = 10#随便取一个批量数量

for X, y  in data_iter(batch_size, features, labels):
   print(X,"\n", y)
   break
    
   
    

In [ ]:
#初始化变量并且定义 y_hat 用于 loss函数 的定义
w = torch.normal(0, 0.01, size = (2,1), requires_grad=True)#初始化 w 并且请求梯度树
b = torch.zeros(1, requires_grad=True)#初始化 b 并且请求梯度树

def linreg(w, X, b):#定义线性回归函数
    return torch.matmul(w, X) + b#线性回归函数返回 y_hat 

In [ ]:
#定义 均方loss函数
def squared_loss(y_hat, y):#主损失函数定义
    return (y_hat - y.reshape(y_hat.shape))**2/2#函数定义为 y_hat - y的平方除以二 

In [ ]:
#定义小批量随机梯度下降
def sgd(params, lr, batch_size):#定义随机梯度下降函数
    with torch.no_grad():#防止自动求导
        for param in params:#遍历参数
            param -= lr*param.grad/batch_size#按照算法更新参数
            param.grad.zero_()#清空梯度

In [ ]:
lr = 0.1#定义学习率
num_epochs = 50#学习遍数
net = linreg
loss = squared_loss

for epoch in range(num_epochs):#遍历样本数
    for X, y in data_iter(batch_size, features, labels):#遍历样本特征和样本标签
        l = loss(net(X, w, b), y)#使用 loss函数 
        l.sum().backward()#对 loss函数 求梯度
        sgd([w, b], lr, batch_size)#进行小批量随机梯度下降
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)#验证集评估模型
        print(f'epoch {epoch +1}, loss {float(train_l.mean()):f}')